# Análisis de Portafolios de Inversión con datos de Yahoo Finance
#### Descargando los datos
Existe una librería llamada yfinance que nos permite descargar series de stocks y rendimientos (https://github.com/ranaroussi/yfinance) de la web Yahoo Finance (https://help.yahoo.com/kb/SLN2311.html)

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

Con la función 'yf.download', podemos descargar varias series al mismo tiempo. 

In [ ]:
tickers = ['GOOGL','AAPL','FB','JPM','BAC','HSBC','GOLD','MRNA','JNJ','PFE','COST','WMT','KR']
data = yf.download(tickers,auto_adjust=True,start='2017-01-01',end='2022-01-01').dropna()
returns,prices = data.Close.pct_change().dropna(),data['Close']
returns.head()

In [ ]:
prices.head()

Podemos visualizar algunas series que nos parezcan interesantes:

In [ ]:
prices['AAPL'].plot(title="Apple stock price")

In [ ]:
prices['GOOGL'].plot(title="Google stock price")

#### Análisis de Portafolios de Inversión
Ahora podemos trabajar con los retornos para encontrar el mix óptimo para invertir (Ejemplo tomado de: https://builtin.com/data-science/portfolio-optimization-python)

In [ ]:
!pip install PyPortfolioOpt

In [ ]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier

mu = mean_historical_return(prices)
S = CovarianceShrinkage(prices).ledoit_wolf()

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(dict(cleaned_weights))

Los pesos resultantes nos dan una idea de cómo es la distribución óptima, utilizando el método básico de Media y Varianza. Podemos calcular la rentabilidad esperada con esta combinación:

In [ ]:
ef.portfolio_performance(verbose=True)

Finalmente, podemos encontrar la distribución óptima de opciones para una inversión de USD 100k:

In [ ]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

latest_prices = get_latest_prices(prices)

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000)

allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

La estrategia sería comprar 232 opciones de Apple, 60 de CostCo, 83 de Moderna y 1 de Google. Además nos muestra el monto que nos queda de resto.

Elaborado por Luis Cajachahua bajo licencia MIT (2022)